# Goal: Trying to understand how the Polymarket API works

In [1]:
import requests

params = {
    "closed": "false",
    "limit": 500,
    "tag_id": 188,
    "related_tags": "false"
}

r = requests.get("https://gamma-api.polymarket.com/events", params=params)
response = r.json()


In [2]:
for event in response:
    print(event['title'])

Trump ends Ukraine war in first 90 days?
Will Trump pardon Hunter Biden?
Trump declassifies UFO files in first 100 days?
Who will be part of Trump's Administration?
Trump ends taxes on tips in first 100 days?
Trump signs national abortion ban?
Who will Trump pardon in first 100 days?
Will Trump end Department of Education in first 100 days?
Will Trump create Bitcoin reserve in first 100 days?
Will Trump jail Hillary Clinton?
Will Trump remove Jerome Powell in first 100 days?
Trump declassifies Diddy list?
Will Trump repeal Presidential term limits?
Trump Admin recommends removing fluoride from water?
Will Trump impose large tariffs in his first 6 months? 
Trump uses a recess appointment for Cabinet?
Which Trump picks will be confirmed?
Trump imposes 40% tariff on China in first 100 days?
US congress stock trading ban in Trump's first 100 days?
Will 3+ Trump cabinet picks fail to get appointed?
Will Trump eliminate daylight saving time in first 100 days?
Will Trump recognize Somaliland 

In [3]:
for market in response[5]['markets']:
    if 'outcomePrices' in market and 'clobTokenIds' in market:
        print(market['id'], market['question'], 'outcomePrices' in market and market['outcomePrices'])
        print('clobTokenIds' in market and market['clobTokenIds'])
        print('=====')

512128 Trump signs national abortion ban? ["0.036", "0.964"]
["51973331945880957072599010054618681509307274170552732113727065440341771479322", "105637525420107465016899790388284587220624374177780103002517436285549579895981"]
=====


In [13]:
import json
import asyncio
import websockets
import datetime
from rich.live import Live
from rich.table import Table
from rich.console import Console

url = 'wss://ws-subscriptions-clob.polymarket.com/ws/market'
console = Console()

# Replace these placeholders with actual asset IDs for the market:
trump_ban_yes_token = "51973331945880957072599010054618681509307274170552732113727065440341771479322"
trump_ban_no_token = "105637525420107465016899790388284587220624374177780103002517436285549579895981"

last_time_pong = datetime.datetime.now()
latest_data = {}

def make_table():
    table = Table(title="Polymarket Order Book: Trump signs national abortion ban?")
    table.add_column("Outcome", justify="center")
    table.add_column("Top 5 Bids", justify="left")
    table.add_column("Top 5 Asks", justify="left")

    for asset_id, data in latest_data.items():
        bids = "\n".join([f"{b['price']} x {b['size']}" for b in data.get("bids", [])[:5]]) or "N/A"
        asks = "\n".join([f"{a['price']} x {a['size']}" for a in data.get("asks", [])[:5]]) or "N/A"
        outcome = "YES" if asset_id == trump_ban_yes_token else "NO"
        table.add_row(outcome, bids, asks)
    return table

async def run():
    global last_time_pong
    async with websockets.connect(url) as websocket:
        await websocket.send(json.dumps({
            "assets_ids": [trump_ban_yes_token, trump_ban_no_token],
            "type": "market"
        }))

        with Live(make_table(), refresh_per_second=1, console=console) as live:
            while True:
                msg = await websocket.recv()
                if msg != "PONG":
                    last_time_pong = datetime.datetime.now()
                else:
                    continue

                data = json.loads(msg)
                if isinstance(data, list):
                    for market_update in data:
                        latest_data[market_update["asset_id"]] = market_update
                        console.log(market_update)  # Print all raw data
                elif isinstance(data, dict):
                    asset_id = data.get("asset_id")
                    if asset_id:
                        latest_data[asset_id] = data
                        console.log(data)  # Print all raw data

                if last_time_pong + datetime.timedelta(seconds=10) < datetime.datetime.now():
                    await websocket.send("PING")

                live.update(make_table())

# If running in Jupyter, use `await run()`
# If running as a script:
await run()


 Polymarket Order Book: Trump signs  
       national abortion ban?        
┏━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Outcome ┃ Top 5 Bids ┃ Top 5 Asks ┃
┡━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│   YES   │ N/A        │ N/A        │
│   NO    │ N/A        │ N/A        │
└─────────┴────────────┴────────────┘

CancelledError: 